## **A) Importações**

In [1]:
#pip install jupyter-dash
#pip install dash

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline

import plotly.express as px
import plotly.graph_objects as go
import plotly.colors as colors

from dash import dash, dcc, html, callback_context
from dash.dependencies import Input, Output, State
import dash.exceptions
import dash_table

from jupyter_dash import JupyterDash

In [3]:
df1 = pd.read_csv("variaveis_totais.csv")
df2 = pd.read_excel("eventos_totais.xlsx")

#### **A.1) Countries info**

In [4]:
var = df1[['country','country_code','indicator', 'indicator_user', 'year','value']]

In [5]:
var.head()

,country,country_code,indicator,indicator_user,year,value
0,Albania,ALB,gdp_growth,GDP Growth (%),1960,NaN
1,Albania,ALB,gdp_growth,GDP Growth (%),1961,NaN
2,Albania,ALB,gdp_growth,GDP Growth (%),1962,NaN
3,Albania,ALB,gdp_growth,GDP Growth (%),1963,NaN
4,Albania,ALB,gdp_growth,GDP Growth (%),1964,NaN


In [6]:
# Encontre os anos mínimos e máximos no dataframe
min_year = var.year.min()
max_year = var.year.max()

#### **A.2) Events info**

In [7]:
df2 = df2[['year_event','country_event','event_type','event']]

In [13]:
ev = df2.query('year_event > 1960')

In [9]:
#ev = ev.reset_index(inplace=True)

In [14]:
ev = ev[['year_event','country_event','event_type','event']].rename(columns={'year_event': 'Year',
                                                                             'country_event': 'Country',
                                                                             'event_type':'Event Type',
                                                                             'event':'Event'})

In [15]:
ev.head()

,Year,Country,Event Type,Event
94,1961.0,Portugal,War/Dispute,"Beginning of the Portuguese colonial war, with..."
95,1961.0,Germany,Geopolitical,Construction began on the Berlin Wall between ...
96,1961.0,Malta,Geopolitical,The State of Malta is created pursuant to the ...
97,1962.0,Albania,Economic,The Albanian regime introduced an austerity pr...
98,1962.0,France,Independence,"End of the Algerian War, Algeria, a French col..."


In [16]:
# Encontre os anos mínimos e máximos no dataframe
min_year_ev = ev['Year'].min()
max_year_ev = ev['Year'].max()

## **B) Visualização em si**

In [52]:
# Criando app
del app

In [53]:
app = JupyterDash(__name__, suppress_callback_exceptions=True)

In [54]:
### PÁGINA
table_style = {
    'fontFamily': 'Arial',
    'borderCollapse': 'collapse',
    'fontSize': '12px',
    'lineHeight': '1.2',
    'marginBottom': '10px',
    'width': '100%',
    'height': '50%',
    'tableLayout': 'auto'
}

cell_style = {
    'border': '1.1px solid #d3d3d3',
    'padding': '6px',
    'textAlign': 'left',
    'backgroundColor': '#FFFFFF'
}

def generate_table(dataframe):
    return html.Table(
        # Header
        [html.Tr([html.Th(col, style=cell_style) for col in dataframe.columns], style=table_style)] +
        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col], style=cell_style) for col in dataframe.columns
        ], style=table_style) for i in range(len(dataframe))],
        style=table_style
    )


# Criando o HTML geral
app.layout = html.Div([

    html.Br(style={"height": '50'}),

    html.Div([
        html.Br(style={"height": '10'}),
    html.Div([
        html.Div([
            html.Img(src='assets/infocountry logo.png',
                     style={'margin-left': '17.5px', 'margin-top': '2.5px', 'float': 'left', 'max-height': '100%'}),
        ], style={'width': '50%', 'height': '80px'}),
        html.Div([
            html.Img(src='assets/uc logo.png',
                     style={'margin-right': '17.5px', 'margin-top': '1px', 'float': 'right', 'max-height': '100%'}),
        ], style={'width': '50%', 'height': '80px'})
    ], style={'width': '100%', 'display': 'flex', 'justify-content': 'space-between'}),
        html.Br(style={"height": '10'})
    ], style={'margin':'auto', 'background-color':'#4B0082'}),
    
    html.Br(style={"height": '20'}),

    html.Div([
        html.Br(style={"height": '5'}),

        html.Div([
            html.H2('Welcome to the country comparator!')
        ], style={'width': '98%', 'margin':'auto'}),

        html.Div([
            html.P('Here you can explore some socioeconomic indicators of European countries since 1960.'),
            html.P(children=[
                'When observed some behavior that calls attention, ',
                html.U('remember that year for the next step'),
                '.'
            ])
        ], style={'width': '98%', 'margin':'auto'}),

        html.Div([
            html.Div([
                html.P("Select up to 5 countries:"),
                dcc.Dropdown(
                    id="dropdown-country",
                    options=[{'label': str(val), 'value': val} for val in sorted(var.country.unique())],
                    value=[],
                    multi=True),
            ], style={'width': '72%', 'display': 'inline-block', 'padding': '0 10'}),

            html.Div([
                html.P("Select 1 indicator:"),
                dcc.Dropdown(
                    id="dropdown-indicator",
                    options=[{'label': val, 'value': val} for val in sorted(var.indicator_user.unique())],
                    value=[],
                    multi=True),
            ], style={'width': '27%', 'display': 'inline-block', 'padding': '0 10'})
        ], style={'width':'98%', 'margin':'auto'}),

        html.Br(style={"height": '10'}),

        html.Div([
            dcc.Graph(id="line"),
            html.Div([
                html.Div([
                    html.Label("Select scale:", style={'display': 'inline-block', 'margin-right': '10px'})
                ], style={'display':'inline-block'}),
                html.Div([
                    dcc.RadioItems(id='radio-scale',
                                   options=[{'label':'Linear', 'value':'linear'},
                                            {'label':'Log', 'value':'log'}],
                                   value='linear',
                                   labelStyle={'display':'inline-block', 'margin-right': '10px'})
                    ], style={'display': 'inline-block'}),
                html.Div(id="scale-buttons", style={'display': 'inline-block'})
            ])
        ], style={'width': '80%', 'margin':'auto'}),
        
        html.Br(style={"height": '10'}),

    ], style={'background-color':'#FFE4E1'}),

    html.Br(style={"height": '20'}),
    html.Hr(),
    html.Br(style={"height": '20'}),

    html.Div([
        html.Br(style={"height": '5'}),

        html.Div([
            html.H2('What relevant events happened?'),
            ], style={'width': '98%', 'margin':'auto'}),

        html.Div([
            html.P(children=[
                'Did any country catch your eyes? Take it as a ',
                html.U('benchmark compared to others'),
                '.'
            ]),
            html.P("Check if there is some historical event around the time of the strange behaviour.")
        ], style={'width': '98%', 'margin':'auto'}),

        html.Div([
            html.Div([
                html.P("Select 1 reference country:"),
                dcc.Dropdown(
                    id="dropdown-ref_country",
                    options=[{'label': str(val), 'value': val} for val in sorted(ev['Country'].unique())],
                    value=[],
                    multi=True)
            ], style={'width': '21%', 'display': 'inline-block', 'padding': '0 10'}),

            html.Div([
                html.P("Select some event type(s):"),
                dcc.Dropdown(
                    id='dropdown-event_type',
                    options=[{'label': 'SELECT ALL', 'value': 'all'}] + [{'label': str(val), 'value': val} for val in sorted(ev['Event Type'].unique())],
                    value=[],
                    multi=True)
            ], style={'width': '78%', 'display': 'inline-block', 'padding': '0 10'})
        ], style= {'width': '98%', 'margin':'auto'}),

        html.Br(style={"height": '10'}),

        html.Div([
            html.Div([
                html.P("Select a range of interest:")
            ], style={'width': '98%', 'margin':'auto'}),
            html.Div([
                dcc.RangeSlider(
                    id='year-slider3',
                    min=min_year_ev,
                    max=max_year_ev,
                    value=[min_year_ev, max_year_ev],
                    marks={str(year): {'label': str(year), 'style': {'transform':'rotate(45deg)'}}
                           if year%2==1 else '' for year in range(int(min_year_ev), int(max_year_ev)+1, 1)},
                    step=None)
            ], style={'width': '98%', 'margin':'auto', 'paddingTop':'15px', 'paddingBottom': '15px', 'background-color':'#FFFFFF'}),

            html.Br(style={"height": '10'}),

            html.Div([
                html.P(f"Main events happened:"),
                html.Div(id='table-container', children=generate_table(ev))
            ], style={'width':'98%', 'margin':'auto'})
        ]),

        html.Br(style={"height": '7.5'}),
    ], style={'background-color':'#E6E6FA'}),

    html.Br(style={"height": '20'}),
    html.Hr(),
    html.Br(style={"height": '20'}),

    html.Div([
        html.Br(style={"height": '5'}),

        html.Div([
            html.H2('Mean impact before and after some event!'),
        ], style={'width': '98%', 'margin':'auto'}),

        html.Div([
            html.P("Finally, let's see if the event brought any change in the variable of the reference country related to others."),            
            html.P(children=[
                'There will appear ',
                html.U('numerical comparison'),
                ' in bars (level) and ',
                html.U('percentage comparison'),
                ' in maps (relational). Two different ways!'
            ])

        
        ], style={'width': '98%', 'margin':'auto'}),

        html.Div([
            html.Div([
                html.P("Enter the reference year:"),
                dcc.Input(
                    id='reference-year',
                    type='number',
                    value=1966, #min_year_ev,
                    style={'width': '100px'}
                    )
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '0 10'}),

            html.Div([
                html.P("Enter interval years:"),
                dcc.Input(
                    id='interval-years',
                    type='number',
                    value=3,
                    style={'width': '100px'}
                    ),
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '0 10'})
        ], style= {'width': '98%', 'margin':'auto'}),

        html.Br(style={"height": '50'}),

        html.Div([
            html.Div([
                dcc.Graph(id="bar1"),
                dcc.Graph(id='map1', figure={})
            ], style={'width':'49.5%', 'display':'inline-block', 'padding':'0 10'}),

            html.Div([
                dcc.Graph(id="bar2"),
                dcc.Graph(id='map2', figure={})
            ], style={'width':'49.5%', 'display':'inline-block', 'padding':'0 10'})
        ], style={'width': '98%', 'margin':'auto'}),

        html.Br(style={"height": '7.5'}),

    ], style={'background-color':'#E0FFFF'}),

    html.Br(style={"height": '100'}),
    
    html.Div([
        html.Div([        
            html.P('Mestrado em Engenharia e Ciência de Dados ● Visualização Avançada'),
            html.P('Andrei Fokin Teixeira #2022135701 ● Rafael Santos Gouveia #2022130518'),
            html.P('InfoCountry©'),
        ], style={'fontSize':'75%', 'line-height': '0.25', 'color': 'white', 'text-align':'center'}),
    ], style={'margin':'auto', 'background-color':'#4B0082', 'padding': '2.5px'}),
    
    html.Br(style={"height": '50'})

], style={'fontFamily':'Arial', 'fontSize':'90%', 'width':'90%', 'margin':'auto', 'align-items':'center'})



#### FUNÇÕES
# Seleciona apenas os primeiros 5 países da lista
@app.callback(Output("dropdown-country", "value"), [Input("dropdown-country", "value")], allow_duplicate=True)
def validate_country_selection(value):
    if len(value) > 5:
        return value[:5]
    return value


# Seleciona apenas o primeiro 1 indicador da lista
@app.callback(Output("dropdown-indicator", "value"), [Input("dropdown-indicator", "value")], allow_duplicate=True)
def validate_indicator_selection(value):
    if len(value) > 1:
        return value[:1]  
    return value


# Primeiro gráfico - Linhas
@app.callback(Output("line", "figure"), 
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("radio-scale", "value")], allow_duplicate=True)
def display_line1(selected_countries, selected_indicators, scale):
    if not selected_countries and not selected_indicators:
        return {}
    filtered_var = var[(var['country'].isin(selected_countries)) & (var['indicator_user'].isin(selected_indicators))]
    colors = [px.colors.qualitative.Plotly[1], px.colors.qualitative.Plotly[0], '#57DB51', '#F4C815', px.colors.qualitative.Plotly[6]]
    line1 = px.line(filtered_var,
                    x='year',
                    y='value',
                    color='country',
                    title='Variables time line',
                    log_y=(scale == "log"),
                    color_discrete_sequence=colors)
    line1.update_layout(title_x=0.5,
                        title_y=0.93,
                        margin=dict(l=50, r=50, t=50, b=50),
                        height=400,
                        yaxis=dict(title='Value', type=("log" if scale == "log" else "linear")),
                        xaxis=dict(title='Year'),
                        legend_title_text='Countries')
    return line1


# Selecionar o país de referencia apenas quando houver algum anteriormente selecionado
@app.callback(Output('dropdown-ref_country', 'options'), Input('dropdown-country', 'value'), allow_duplicate=True)
def update_ref_country_options(selected_countries):
    if not selected_countries:
        return []
    options = [{'label': str(val), 'value': val} for val in sorted(ev['Country'].unique())]
    options = [opt for opt in options if opt['value'] in selected_countries]
    return options


# Escolhe o primeiro da lista acima
@app.callback(Output("dropdown-ref_country", "value"), [Input("dropdown-ref_country", "value")], allow_duplicate=True)
def validate_ref_country_selection(value):
    if len(value) > 1:
        return value[:1]
    return value


## Seletor de eventos
@app.callback(Output('dropdown-event_type', 'value'),
               [Input('dropdown-event_type', 'value'), Input('dropdown-event_type', 'options')],
               allow_duplicate=True)
def update_event_type(value, options):
    ctx = dash.callback_context
    if not ctx.triggered:
        button_id = 'None'
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    if button_id == 'dropdown-event_type':
        if len(value) > 14:
            return value[:14]
        return value
    elif button_id == 'dropdown-event_type':
        return [option['value'] for option in options] if 'all' in [option['value'] for option in options] else []
    else:
        return dash.no_update

    
## Tabela
@app.callback(Output('table-container', 'children'),
               [Input('dropdown-ref_country','value'), Input('dropdown-event_type','value'), Input('year-slider3', 'value'),
                Input('dropdown-event_type', 'options')],
               allow_duplicate=True)
def update_table(ref_country, event_type, year_range, event_type_options):
    # Verificar se "SELECT ALL" está selecionado
    if 'all' in event_type:
        # Obter todos os valores de evento disponíveis, exceto "all"
        event_type_values = [option['value'] for option in event_type_options if option['value'] != 'all']
        # Usar todos os valores de evento disponíveis se nenhum evento individual tiver sido selecionado
        event_type = event_type_values if len(event_type) == 0 else event_type_values
    # Filtrar o dataframe original utilizando os valores selecionados nos dropdowns e no RangeSlider
    filtered_df_ev = ev[(ev['Country'].isin(ref_country)) & 
                       (ev['Event Type'].isin(event_type)) & 
                       (ev['Year'].between(year_range[0], year_range[1]))]
    # Chamar a função generate_table para criar a tabela HTML
    table_ev = generate_table(filtered_df_ev)
    # Retornar a tabela HTML
    return table_ev


## Gráficos do lado esquerdo
# Barra 1
@app.callback(Output("bar1", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("reference-year", "value"), 
               Input("interval-years", "value"), Input("dropdown-ref_country", "value")],
               allow_duplicate=True)
def display_bar1(selected_countries, selected_indicators, reference_year, interval_years, reference_country):
    if not reference_country:
        return {}
    # Verificar se o valor de "reference_year" é numérico
    if not isinstance(reference_year, (int, float)):
        return {}
    # Converter o valor de "reference_year" para inteiro
    reference_year = int(reference_year)
    # Identificar se cada linha corresponde ao país de referência
    filtered_var = var[(var['country'].isin(selected_countries)) &
                       (var['indicator_user'].isin(selected_indicators)) &
                       (var['year'] >= reference_year - int(interval_years)) &
                       (var['year'] <= reference_year)]
    # Verificar se a coluna "is_reference" existe no dataframe
    if 'is_reference' not in filtered_var.columns:
        filtered_var['is_reference'] = 0
    # Atualizar os valores da coluna "is_reference" para o país de referência
    filtered_var.loc[filtered_var['country'] == str(reference_country[0]), 'is_reference'] = 1
    # "Novo" dataframe
    grouped_var = round(filtered_var.groupby(['country', 'indicator_user']).mean(), 2).reset_index()
    # Ordenar o dataframe para a referência ser a primeira barra
    grouped_var = grouped_var.sort_values(by=['is_reference', 'value'], ascending=[False, True])
    # Criando um dicionário com as cores para cada país
    colors = {str(reference_country[0]): '#ED503E'}
    for country in grouped_var['country']:
        if country != str(reference_country[0]):
            colors[country] = '#A3BAC7'
    # Criando gráfico e definindo as cores
    bar1 = px.bar(grouped_var,
                  x='country',
                  y='value',
                  text='value',
                  color='country',
                  color_discrete_map=colors,
                  orientation='v',
                  title="Mean value - BEFORE year reference")
    bar1.update_layout(title_x=0.5,
                       title_y=0.93,
                       margin=dict(l=50, r=50, t=50, b=50),
                       height=400,
                       yaxis=dict(title='Mean value'),
                       xaxis=dict(title='Country'),
                       showlegend=False)
    return bar1


# Mapa 1
@app.callback(Output('map1', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("reference-year", "value"), 
               Input("interval-years", "value"), Input("dropdown-ref_country", "value")],
               allow_duplicate=True)
def display_map1(selected_countries, selected_indicators, reference_year, interval_years, reference_country):
    if not selected_countries or not reference_country:
        return {}
    filtered_var = var[var['country'].isin(selected_countries) &
                       (var['indicator_user'].isin(selected_indicators)) &
                       (var['year'] >= int(reference_year)-int(interval_years)) &
                       (var['year'] <= int(reference_year))]
    # Cria um novo DataFrame apenas com as informações do país de referência selecionado
    ref_country_var = filtered_var[filtered_var['country'] == str(reference_country[0])]
    # Calcula a média dos valores para o país de referência selecionado
    ref_country_mean = round(ref_country_var['value'].mean(),2)
    # Cria uma nova coluna com o valor percentual em relação ao país de referência selecionado
    xxx = (round(filtered_var.groupby(['country', 'indicator_user', 'country_code'])['value'].mean(), 2).reset_index())
    # Add valores percentuais ao dataframe de comparação
    xxx['value_pct'] = (xxx['value']/ref_country_mean - 1) * 100
    # Verifica o sinal de ref_country_mean
    sign_ref_country_mean = np.sign(ref_country_mean)
    # Aplica a condição nos valores de xxx['value_pct']
    xxx['value_pct'] = xxx['value_pct'] * sign_ref_country_mean
    # Criando mapa com a cor cinza para os valores nulos ou quando xxx está vazio
    if xxx['value_pct'].isnull().all():
        map1 = px.choropleth(xxx,
                             locations="country_code",
                             color="value_pct",
                             locationmode='ISO-3',
                             hover_name="country",
                             custom_data=["value_pct"],
                             projection="kavrayskiy7",
                             scope='europe',
                             color_continuous_scale=['darkgray','darkgray'],
                             color_continuous_midpoint=0,
                             labels={'value_pct': 'Variation (%)'})
        gray_map1 = px.choropleth(filtered_var,
                                  locations="country_code",
                                  color_discrete_sequence=['darkgray'],
                                  locationmode='ISO-3',
                                  hover_name="country",
                                  projection="kavrayskiy7",
                                  scope='europe')
        map1.add_trace(gray_map1.data[0])
        map1.update_traces(hovertemplate='<b>%{hovertext}</b><br>NO DATA')
    else:
        map1 = px.choropleth(xxx,
                             locations="country_code",
                             color="value_pct",
                             locationmode='ISO-3',
                             hover_name="country",
                             custom_data=["value_pct"],
                             projection="kavrayskiy7",
                             scope='europe',
                             color_continuous_scale=['#B22222', '#F58600', '#FFFF00', '#57B300', '#006400'],#'RdYlGn',
                             color_continuous_midpoint=0,
                             labels={'value_pct': 'Variation (%)'})
        map1.update_traces(hovertemplate='<b>%{hovertext}</b><br>Variation: %{customdata[0]:.2f}%')
    map1.update_traces(colorbar={'title': {'text': 'Variation (%)'}})
    # Personalizar o layout do mapa
    map1.update_layout(geo=dict(scope='europe',
                                showland=True,
                                landcolor='#f2f2f2',
                                showocean=True,
                                oceancolor='#a6bddb',
                                showcountries=True,
                                countrycolor='gray',
                                showsubunits=True,
                                subunitcolor='gray',
                                projection=dict(type='kavrayskiy7')),
                                title = {'text': "Percentual variation related to " + str(reference_country[0]) + "<br>BEFORE year reference",
                                         'x': 0.5,
                                         'y': 0.9,
                                         'xanchor': 'center',
                                         'yanchor': 'top'},
                                showlegend=False)
    return map1


## Gráficos do lado direito
# Barras 2
@app.callback(Output("bar2", "figure"),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("reference-year", "value"), 
               Input("interval-years", "value"), Input("dropdown-ref_country", "value")],
               allow_duplicate=True)
def display_bar2(selected_countries, selected_indicators, reference_year, interval_years, reference_country):
    if not reference_country:
        return {}
    # Verificar se o valor de "reference_year" é numérico
    if not isinstance(reference_year, (int, float)):
        return {}
    # Converter o valor de "reference_year" para inteiro
    reference_year = int(reference_year)
    # Identificar se cada linha corresponde ao país de referência
    filtered_var = var[(var['country'].isin(selected_countries)) &
                       (var['indicator_user'].isin(selected_indicators)) &
                       (var['year'] >= reference_year) &
                       (var['year'] <= reference_year + int(interval_years))]
    # Verificar se a coluna "is_reference" existe no dataframe
    if 'is_reference' not in filtered_var.columns:
        filtered_var['is_reference'] = 0
    # Atualizar os valores da coluna "is_reference" para o país de referência
    filtered_var.loc[filtered_var['country'] == str(reference_country[0]), 'is_reference'] = 1
    # "Novo" dataframe
    grouped_var = round(filtered_var.groupby(['country', 'indicator_user']).mean(), 2).reset_index()
    # Ordenar o dataframe para a referência ser a primeira barra
    grouped_var = grouped_var.sort_values(by=['is_reference', 'value'], ascending=[False, True])
    # Criando um dicionário com as cores para cada país
    colors = {str(reference_country[0]): '#ED503E'}
    for country in grouped_var['country']:
        if country != str(reference_country[0]):
            colors[country] = '#A3BAC7'
    # Criando gráfico e definindo as cores
    bar2 = px.bar(grouped_var,
                  x='country',
                  y='value',
                  text='value',
                  color='country',
                  color_discrete_map=colors,
                  orientation='v',
                  title="Mean value - AFTER year reference")
    bar2.update_layout(title_x=0.5,
                       title_y=0.93,
                       margin=dict(l=50, r=50, t=50, b=50),
                       height=400,
                       yaxis=dict(title='Mean value'),
                       xaxis=dict(title='Country'),
                       showlegend=False)
    return bar2


# Mapa 2
@app.callback(Output('map2', 'figure'),
              [Input("dropdown-country", "value"), Input("dropdown-indicator", "value"), Input("reference-year", "value"), 
               Input("interval-years", "value"), Input("dropdown-ref_country", "value")],
               allow_duplicate=True)
def display_map2(selected_countries, selected_indicators, reference_year, interval_years, reference_country):
    if not selected_countries or not reference_country:
        return {}    
    filtered_var = var[var['country'].isin(selected_countries) &
                       (var['indicator_user'].isin(selected_indicators)) &
                       (var['year'] >= int(reference_year)) &
                       (var['year'] <= int(reference_year)+int(interval_years))]
    # Cria um novo DataFrame apenas com as informações do país de referência selecionado
    ref_country_var = filtered_var[filtered_var['country'] == str(reference_country[0])]
    # Calcula a média dos valores para o país de referência selecionado
    ref_country_mean = round(ref_country_var['value'].mean(),2)
    # Cria uma nova coluna com o valor percentual em relação ao país de referência selecionado
    xxx = (round(filtered_var.groupby(['country', 'indicator_user', 'country_code'])['value'].mean(), 2).reset_index())
    # Add valores percentuais ao dataframe de comparação
    xxx['value_pct'] = (xxx['value']/ref_country_mean - 1) * 100
    # Verifica o sinal de ref_country_mean
    sign_ref_country_mean = np.sign(ref_country_mean)
    # Aplica a condição nos valores de xxx['value_pct']
    xxx['value_pct'] = xxx['value_pct'] * sign_ref_country_mean
    # Criando mapa com a cor cinza para os valores nulos ou quando xxx está vazio
    if xxx['value_pct'].isnull().all():
        map2 = px.choropleth(xxx,
                             locations="country_code",
                             color="value_pct",
                             locationmode='ISO-3',
                             hover_name="country",
                             custom_data=["value_pct"],
                             projection="kavrayskiy7",
                             scope='europe',
                             color_continuous_scale=['darkgray','darkgray'],
                             color_continuous_midpoint=0,
                             labels={'value_pct': 'Variation (%)'})
        gray_map2 = px.choropleth(filtered_var,
                                  locations="country_code",
                                  color_discrete_sequence=['darkgray'],
                                  locationmode='ISO-3',
                                  hover_name="country",
                                  projection="kavrayskiy7",
                                  scope='europe')
        map2.add_trace(gray_map2.data[0])
        map2.update_traces(hovertemplate='<b>%{hovertext}</b><br>NO DATA')
    else:
        map2 = px.choropleth(xxx,
                             locations="country_code",
                             color="value_pct",
                             locationmode='ISO-3',
                             hover_name="country",
                             custom_data=["value_pct"],
                             projection="kavrayskiy7",
                             scope='europe',
                             color_continuous_scale=['#B22222', '#F58600', '#FFFF00', '#57B300', '#006400'],#'RdYlGn',
                             color_continuous_midpoint=0,
                             labels={'value_pct': 'Variation (%)'})
        map2.update_traces(hovertemplate='<b>%{hovertext}</b><br>Variation: %{customdata[0]:.2f}%')
    map2.update_traces(colorbar={'title': {'text': 'Variation (%)'}})
    # Personalizar o layout do mapa
    map2.update_layout(geo=dict(scope='europe',
                                showland=True,
                                landcolor='#f2f2f2',
                                showocean=True,
                                oceancolor='#a6bddb',
                                showcountries=True,
                                countrycolor='gray',
                                showsubunits=True,
                                subunitcolor='gray',
                                projection=dict(type='kavrayskiy7')),
                                title = {'text': "Percentual variation related to " + str(reference_country[0]) + "<br>BEFORE year reference",
                                         'x': 0.5,
                                         'y': 0.9,
                                         'xanchor': 'center',
                                         'yanchor': 'top'},
                                showlegend=False)
    return map2


### RODAR TUDO
if __name__ == "__main__":
    app.run_server(port=8051, debug=True)

Dash is running on http://127.0.0.1:8051/

Dash app running on http://127.0.0.1:8051/
